In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')
#%cd /content/drive/MyDrive/The-Role-of-Spatial-Context-in-Deep-Learning-based-Semantic-Segmentation-of-Remote-Sensing-Imagery/dfc20/utilities
#!pip install rasterio

In [ ]:
#!cp -r /content/drive/MyDrive/The-Role-of-Spatial-Context-in-Deep-Learning-based-Semantic-Segmentation-of-Remote-Sensing-Imagery/dfc20/data /content/data

In [12]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
from matplotlib.patches import Patch
import json
from collections import Counter

#parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
#sys.path.insert(0, parent_dir)

from dataset import DFC20

In [ ]:
#path = "../data"
#path = "/content/data"
path = "./data"

# load datasets
ds = DFC20(path,
            subset="train",
            use_s2_RGB=False,
            use_s2_hr=False,
            use_s2_all=True)

[Load]: 100%|██████████| 1160/1160 [00:00<00:00, 289004.61it/s]

loaded 1160 samples from the DFC20 subset test


In [5]:
img = ds.__getitem__(1)['image']
print(img.shape)
sum_img = np.sum(img, axis=(1, 2))  # Sum over height and width (axis 1 and 2)
sum_sq_img = np.sum(img**2, axis=(1, 2))  # Sum of squared pixel values
sum_img

(13, 256, 256)


array([89746592., 71175560., 55347008., 46183528., 44025788., 42829112.,
       42909832., 37724452., 41580960., 16012971.,   406232., 33433124.,
       28542206.], dtype=float32)

MEAN STD

In [ ]:
sum_img = np.zeros((13,))  # Running sum for each band
sum_sq_img = np.zeros((13,))  # Running sum of squares for each band
num_pixels = 0  # Total number of pixels processed


for i in range(len(ds)):
    img = ds.__getitem__(i)['image']
    img = img.astype(np.float64)

    sum_img += np.sum(img, axis=(1, 2))  # Sum over height (axis 1) and width (axis 2)
    sum_sq_img += np.sum(img**2, axis=(1, 2))  # Sum of squared values for each band

    num_pixels += img.shape[1] * img.shape[2]  # 256 * 256

mean = sum_img / num_pixels  # Mean for each band
std = np.sqrt((sum_sq_img / num_pixels) - mean**2)  # Standard deviation for each band

print("Mean:", mean)
print("Std:", std)

# Save mean and std for later use
np.save("utilities/s2_train_mean.npy", mean)
np.save("utilities/s2_train_std.npy", std)

Mean: [1272.16720372 1027.73928382  937.9851055   779.450488   1024.74893655
 1767.03042979 2068.67777491 2022.92672832 2234.29554739  788.00590771
   10.91993568 1498.87448473  934.60372392]
Std: [ 197.73851336  261.65425257  300.75439479  468.21129954  487.37184567
  967.7635599  1221.23601915 1230.64561262 1352.95229777  520.38536375
    4.60888687 1016.39034628  772.87821907]


MIN MAX

In [ ]:
# Initialize min and max values for each band
min_img = np.full((13,), np.inf)  # Start with large values
max_img = np.full((13,), -np.inf)  # Start with small values

for i in range(len(ds)):
    img = ds.__getitem__(i)['image']  # Get image (shape: 13, 256, 256)

    # Compute per-band min and max
    min_img = np.minimum(min_img, np.min(img, axis=(1, 2)))
    max_img = np.maximum(max_img, np.max(img, axis=(1, 2)))

print("Min per band:", min_img)
print("Max per band:", max_img)

LABEL MULTI CLASS

In [ ]:
image_class_presence = []

for i in range(len(ds)):
    mask = ds.__getitem__(i)['label']
    class_ids = np.unique(mask)
    image_class_presence.append([int(cls_id) for cls_id in class_ids])

with open("utilities/train_multi_class_per_image.json", "w") as f:
    json.dump(image_class_presence, f)

LABEL MAJORITY CLASS

In [ ]:
majority_classes = []

for i in range(len(ds)):
    mask = ds.__getitem__(i)['label']
    class_counts = Counter(mask.flatten())
    majority_class = class_counts.most_common(1)[0][0]  # Get class with most pixels
    majority_classes.append(int(majority_class))

with open("utilities/train_majority_class_per_image.json", "w") as f:
    json.dump(majority_classes, f)